In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from operator import itemgetter

import networkx as nx

from tqdm import tqdm

Loading the adjacency matrix and the sampled data

In [ ]:
adjacency_matrix = np.load('adjacency_matrix.npy')

data_recipe = pd.read_json('processed_data.json').reset_index(drop=True)

In [ ]:
data_recipe.head(5)

Plotting the adjacency matrix to see sparsity pattern

In [ ]:
plt.figure(figsize=(5,5))
plt.spy(adjacency_matrix)
plt.show()

In [ ]:
def describe_graph(G):
    """
    Helper function for printing various graph properties.
    
    Parameters
    ----------
    
    G: NetworkX graph object
        Graph
    """
    print(nx.info(G))
    if nx.is_connected(G):
        print("Avg. Shortest Path Length: %.4f" %nx.average_shortest_path_length(G))
        print("Diameter: %.4f" %nx.diameter(G)) # Longest shortest path
    else:
        print("Graph is not connected")
        print("Diameter and Avg shortest path length are not defined!")
        
        # Number of connected components.
        comp = list(nx.connected_components(G))
        print('The graph contains', len(comp), 'connected components')
        
        # Largest connected components properties.
        largest_comp = max(comp, key=len)
        percentage_lcc = len(largest_comp)/G.number_of_nodes() * 100
        print('The largest component has', len(largest_comp), 'nodes', 'accounting for %.2f'% percentage_lcc, '% of the nodes')
        lcc_G = G.subgraph(largest_comp)
        print("The diameter of the largest connected component is", nx.diameter(lcc_G))
        print("The avg shortest path length of the largest connected component is", nx.average_shortest_path_length(lcc_G))
        
        # Smallest connected component properties.
        smallest_comp = min(comp, key=len)
        percentage_scc = len(smallest_comp)/G.number_of_nodes() * 100
        print('The smallest component has', len(smallest_comp), 'nodes', 'accounting for %.2f'% percentage_scc, '% of the nodes')
        scc_G = G.subgraph(smallest_comp)
        print("The diameter of the smallest connected component is", nx.diameter(scc_G))
        print("The avg shortest path length of the smallest connected component is", nx.average_shortest_path_length(scc_G))
        
    print("Sparsity: %.4f" %nx.density(G))  # #edges/#edges-complete-graph
    # #closed-triplets(3*#triangles)/#all-triplets
    print("Global clustering coefficient aka Transitivity: %.4f" %nx.transitivity(G))

In [ ]:
graph = nx.from_numpy_array(adjacency_matrix)

In [ ]:
# Add node attributes.
nx.set_node_attributes(graph, data_recipe['title'].to_dict(), 'title')
nx.set_node_attributes(graph, data_recipe['protein'].to_dict(), 'protein')
nx.set_node_attributes(graph, data_recipe['fat'].to_dict(), 'fat')
nx.set_node_attributes(graph, data_recipe['sodium'].to_dict(), 'sodium')
nx.set_node_attributes(graph, data_recipe['calories'].to_dict(), 'calories')
nx.set_node_attributes(graph, data_recipe['rating'].to_dict(), 'rating')
nx.set_node_attributes(graph, data_recipe['ingredients'].to_dict(), 'ingredients')

In [ ]:
describe_graph(graph)

Most of the nodes are in the giant/largest component, which is often the case in a graph.

In [ ]:
# Which nodes/recipes contain the most common ingredients.
degrees = dict(graph.degree(graph.nodes()))
sorted_degree = sorted(degrees.items(), key=itemgetter(1), reverse=True)

# The top 5 most connected nodes/recipes.
for recipe, degree in sorted_degree[:5]:
    print('Node', recipe, 'which is', graph.node[recipe]['title'], 'has common ingredients with', degree, 'recipes:', graph.node[recipe]['ingredients'], '\n')

We can see that that the recipes which contain the most common ingredients are:
- Spicy Shrimp and Bell Pepper Stew with Cumin and Oregano
- Marsala and Dried-Fig Crostata
- Wilted Red Cabbage with Balsamic Vinegar
- Apple Walnut Upside-Down Cake with Calvados Caramel Sauce
- Apricot-Sour Cream Scones

By looking at their ingredients one can conclude that among the most common ingredients we can name:
- egg
- water
- sugar
- butter
- cream

We can see that the most connected recipe nodes are from different categories. 3 are desserts and the other 2 are proper main dishes. This is reasonable, however, as the ingredients we mentioned above tend to be used in most types of recipes and across different categories. Hence, because these recipe nodes each has a subset of them, they ended up being hubs.


In [ ]:
def plot_degree_distribution(G):
    """
    Helper function for plotting the degree distribution of the graph.
    
    Parameters
    ----------
    
    G: NetworkX graph object
        Graph
    """
    degrees = {}
    for node in G.nodes():
        degree = G.degree(node)
        if degree not in degrees:
            degrees[degree] = 0
        degrees[degree] += 1
    sorted_degree = sorted(degrees.items())
    deg = [k for (k,v) in sorted_degree]
    cnt = [v for (k,v) in sorted_degree]
    fig, ax = plt.subplots(figsize=(28,10))
    plt.bar(deg, cnt, width=0.80, color='b')
    plt.title("Degree Distribution")
    plt.ylabel("Frequency")
    plt.xlabel("Degree")
    ax.set_xticks([d+0.05 for d in deg])
    ax.set_xticklabels(deg)

In [ ]:
plot_degree_distribution(graph)

The degree distribution of the network seems to follow a power law. This means that most recipes are connected to the other recipes (have common ingredients), but there are a small number of "hubs", i.e. recipes which are connected to many other recipes because contain a large number of the most basic, common ingredients across different types of cuisines.

In [ ]:
# Clustering coefficient of all nodes (in a dictionary)
clust_coefficients = nx.clustering(graph)

# Average clustering coefficient
avg_clust = sum(clust_coefficients.values()) / len(clust_coefficients)

print(avg_clust)

# Or use directly the built-in method
print(nx.average_clustering(graph))